In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
!pip install pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [13]:
from datasets import load_dataset

df = load_dataset("squad", split="train").to_pandas()
df = df[["title", "context"]]
df = df.drop_duplicates(subset="context")
df

,title,context
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha..."
5,University_of_Notre_Dame,"As at most other universities, Notre Dame's st..."
10,University_of_Notre_Dame,The university is the major seat of the Congre...
15,University_of_Notre_Dame,The College of Engineering was established in ...
20,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...
...,...,...
87574,Kathmandu,"Institute of Medicine, the central college of ..."
87579,Kathmandu,Football and Cricket are the most popular spor...
87584,Kathmandu,The total length of roads in Nepal is recorded...
87589,Kathmandu,The main international airport serving Kathman...


In [14]:
import pinecone
pinecone.init(
    api_key="71065eff-7f95-48c2-a3e1-4d2ef782a8a8",
    environment="eu-west4-gcp"
)

In [15]:
index_name = "extractive-question-answering"


if index_name not in pinecone.list_indexes():

    pinecone.create_index(
        index_name,
        dimension=384,
        metric="cosine"
    )
index = pinecone.Index(index_name)

In [16]:
import torch
from sentence_transformers import SentenceTransformer


device = 0 if torch.cuda.is_available() else -1

retriever = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [17]:
from tqdm.auto import tqdm


batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    
    i_end = min(i+batch_size, len(df))
    batch = df.iloc[i:i_end]
    emb = retriever.encode(batch["context"].tolist()).tolist()
    meta = batch.to_dict(orient="records")
    ids = [f"{idx}" for idx in range(i, i_end)]
    to_upsert = list(zip(ids, emb, meta))
    _ = index.upsert(vectors=to_upsert)

index.describe_index_stats()

  0%|          | 0/296 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 18891}},
 'total_vector_count': 18891}

In [18]:
from transformers import pipeline

model_name = "deepset/electra-base-squad2"
reader = pipeline(tokenizer=model_name, model=model_name, task="question-answering", device=device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
def get_context(question, top_k):
    xq = retriever.encode([question]).tolist()
    xc = index.query(xq, top_k=top_k, include_metadata=True)
    c = [x["metadata"]["context"] for x in xc["matches"]]
    return c

question = "How much oil is Egypt producing in a day?"
context = get_context(question, top_k = 1)
context

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of natural gas (in 2013), which makes Egypt as the largest oil producer not member of the Organization of the Petroleum Exporting Countries (OPEC) and the second-largest dry natural gas producer in Africa. In 2013, Egypt was the largest consumer of oil and natural gas in Africa, as more than 20% of total oil consumption and more than 40% of total dry natural gas consumption in Africa. Also, Egypt possesses the largest oil refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is currently planning to build its first nuclear power plant in El Dabaa city, northern Egypt.']

In [20]:
from pprint import pprint


def extract_answer(question, context):
    results = []
    for c in context:
        
        answer = reader(question=question, context=c)
        
        answer["context"] = c
        results.append(answer)
    
    sorted_result = pprint(sorted(results, key=lambda x: x["score"], reverse=True))
    return sorted_result

extract_answer(question, context)

[{'answer': '691,000 bbl/d',
  'context': 'Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of '
             'natural gas (in 2013), which makes Egypt as the largest oil '
             'producer not member of the Organization of the Petroleum '
             'Exporting Countries (OPEC) and the second-largest dry natural '
             'gas producer in Africa. In 2013, Egypt was the largest consumer '
             'of oil and natural gas in Africa, as more than 20% of total oil '
             'consumption and more than 40% of total dry natural gas '
             'consumption in Africa. Also, Egypt possesses the largest oil '
             'refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is '
             'currently planning to build its first nuclear power plant in El '
             'Dabaa city, northern Egypt.',
  'end': 33,
  'score': 0.9999852180480957,
  'start': 20}]


In [21]:
question = "What are the first names of the men that invented youtube?"
context = get_context(question, top_k=1)
extract_answer(question, context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'answer': 'Hurley and Chen',
  'context': 'According to a story that has often been repeated in the media, '
             'Hurley and Chen developed the idea for YouTube during the early '
             'months of 2005, after they had experienced difficulty sharing '
             "videos that had been shot at a dinner party at Chen's apartment "
             'in San Francisco. Karim did not attend the party and denied that '
             'it had occurred, but Chen commented that the idea that YouTube '
             'was founded after a dinner party "was probably very strengthened '
             'by marketing ideas around creating a story that was very '
             'digestible".',
  'end': 79,
  'score': 0.9999276399612427,
  'start': 64}]


In [22]:
question = "What is Taylor Swift famous for?"
context = get_context(question, top_k=1)
extract_answer(question, context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'answer': 'selfless',
  'context': 'In September 2010, West wrote a series of apologetic tweets '
             'addressed to Swift including "Beyonce didn\'t need that. MTV '
             "didn't need that and Taylor and her family friends and fans "
             'definitely didn\'t want or need that" and concluding with "I\'m '
             'sorry Taylor." He also revealed he had written a song for Swift '
             'and if she did not accept the song, he would perform it himself. '
             'However, on November 8, 2010, in an interview with a Minnesota '
             'radio station, he seemed to recant his past apologies by '
             'attempting to describe the act at the 2009 awards show as '
             '"selfless" and downgrade the perception of disrespect it '
             'created. In "Famous," a track from his 2016 album The Life of '
             "Pablo, West implies that this incident led to Swift's stardom, "
             'rapping, "I feel like me and Taylor 

In [23]:
question = "Who was the first person to step foot on the moon?"
context = get_context(question, top_k=3)
extract_answer(question, context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'answer': 'Armstrong',
  'context': 'The trip to the Moon took just over three days. After achieving '
             'orbit, Armstrong and Aldrin transferred into the Lunar Module, '
             'named Eagle, and after a landing gear inspection by Collins '
             'remaining in the Command/Service Module Columbia, began their '
             'descent. After overcoming several computer overload alarms '
             'caused by an antenna switch left in the wrong position, and a '
             'slight downrange error, Armstrong took over manual flight '
             'control at about 180 meters (590 ft), and guided the Lunar '
             'Module to a safe landing spot at 20:18:04 UTC, July 20, 1969 '
             '(3:17:04 pm CDT). The first humans on the Moon would wait '
             'another six hours before they ventured out of their craft. At '
             '02:56 UTC, July 21 (9:56 pm CDT July 20), Armstrong became the '
             'first human to set foot on the Moon.',

In [24]:
 pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 75.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 85.0 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.8 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=b558699d98a1ffd9b6d13cda6d9bd9fb62f997e70dac8c911b9b6536a9a98de9
  Stored in directory: /root/.cache/pip/wheels/3d/7c/f1/f87f51d07a2513aa0fb128d5eb504da1fcef9c87475d9eef64
Successfully built validators
Note: you may need to restart the kernel to use updated packages.


In [25]:
%%writefile my_app.py

import streamlit as st
import pinecone
import streamlit as st
from transformers import pipeline
from sentence_transformers import SentenceTransformer

PINECONE_KEY = st.secrets["71065eff-7f95-48c2-a3e1-4d2ef782a8a8"]  

@st.experimental_singleton
def init_pinecone():
    pinecone.init(api_key=71065eff-7f95-48c2-a3e1-4d2ef782a8a8, environment="us-west1-gcp")  
    return pinecone.Index("extractive-question-answering")
    
@st.experimental_singleton
def init_models():
    retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
    model_name = 'deepset/electra-base-squad2'
    reader = pipeline(tokenizer=model_name, model=model_name, task='question-answering')
    return retriever, reader

st.session_state.index = init_pinecone()
retriever, reader = init_models()


def card(title, context, score):
    return st.markdown(f"""
    <div class="container-fluid">
        <div class="row align-items-start">
             <div  class="col-md-12 col-sm-12">
                 <b>{title}</b>
                 <br>
                 <span style="color: #808080;">
                     <small>{context}</small>
                     [<b>Score: </b>{score}]
                 </span>
             </div>
        </div>
     </div>
        """, unsafe_allow_html=True)

st.title("")

st.write("""
# Extractive Question Answering
Ask me a question!
""")

st.markdown("""
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.0.0/dist/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
""", unsafe_allow_html=True)

def run_query(query):
    xq = retriever.encode([query]).tolist()
    try:
        xc = st.session_state.index.query(xq, top_k=3, include_metadata=True)
    except:
        # force reload
        pinecone.init(api_key=PINECONE_KEY, environment="eu-west4-gcp")
        st.session_state.index = pinecone.Index("extractive-question-answering")
        xc = st.session_state.index.query(xq, top_k=3, include_metadata=True)

    results = []
    for match in xc['matches']:
        answer = reader(question=query, context=match["metadata"]['context'])
        answer["title"] = match["metadata"]['title']
        answer["context"] = match["metadata"]['context']
        results.append(answer)

    sorted_result = sorted(results, key=lambda x: x['score'], reverse=True)

    for r in sorted_result:
        answer = r["answer"]
        context = r["context"].replace(answer, f"<mark>{answer}</mark>")
        title = r["title"].replace("_", " ")
        score = round(r["score"], 4)
        card(title, context, score)

query = st.text_input("Search!", "")

if query != "":
    run_query(query)

Writing my_app.py


In [26]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2023-04-28 10:38:26--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  2.61MB/s    in 12s     

2023-04-28 10:38:39 (1.11 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]



In [27]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [28]:
get_ipython().system_raw('./ngrok http 8501 &')

In [29]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://34f7-34-151-77-208.ngrok.io


In [ ]:
!streamlit run ./my_app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.19.2.2:8501
  External URL: http://34.151.77.208:8501

